In [1]:
import sys
sys.path.append('../../code/')
import reframed
from pathlib import Path
import pandas as pd
import utils
from dFBA_reframed import dFBA, get_leaky_metabolites, estimate_shadow_prices
import numpy as np
import argparse
from collections import defaultdict

# Small pilot snippets for dFBA of 4 species MWF

In [2]:
FN_MET_IDS = Path('../../Data/met_info_3.csv')
df_met_info = pd.read_csv(FN_MET_IDS)

In [3]:
df_met_info['Metabolite'] = df_met_info['Metabolite'].str.lower()
cs_to_bigg_dict = df_met_info.set_index('Metabolite')['Metabolite id'].to_dict()
df_met_info['log10(Intracelll. conc [mM])'] = np.log10(df_met_info['Intracellular concentration [M]']*1e3)
temp_dict = df_met_info.set_index('Metabolite id')['log10(Intracelll. conc [mM])'].to_dict()
mean_log_conc = df_met_info['log10(Intracelll. conc [mM])'].mean()
bigg_to_log_intra_conc_dict = defaultdict(lambda: mean_log_conc)
for key, value in temp_dict.items():
    if not np.isnan(value):
        bigg_to_log_intra_conc_dict[key] = value

met_id_to_class = df_met_info.set_index('Metabolite id')['Compound class'].to_dict()

## Parametrize Vmax and Km for all species and all cs


In [4]:
FN_GROWTH_PARAMETERS = Path('../../Data/mwf/fitted_growth_parameters.csv')
FN_EXCHANGED_METS = Path('../../Data/mwf/exchanged_mets.csv')
species = ['At', 'Ct', 'Ml', 'Oa']
ratio_of_optimum = 0.9


In [5]:
FN_CARBON_SOURCES = Path('../../Data/mwf/carbon_sources.csv')

In [6]:
df_cs = pd.read_csv(FN_CARBON_SOURCES, index_col=0)
df_cs['Metabolite name'] = df_cs['Metabolite name'].str.lower()


In [7]:
df_cs

,Metabolite name,Class,Formula,C atoms,Molecular weight,Concentration (mM),Bigg ID
Single or pair,,,,,,,
Single,acetate,Organic acid,NaC2H3O2,2.0,82.03430,45.0,ac
Single,d-glucose,Simple sugar,C6H12O6,6.0,180.15588,15.0,glc__D
Single,citrate,Organic acid,C6H8O7,6.0,192.12400,15.0,cit
Single,glutarate,Organic acid,C5H8O4,5.0,132.11000,18.0,glutar
Single,glycerol,Alcohol,C3H8O3,3.0,92.09382,30.0,glyc
Single,inosine,Nucleoside,C10H12N4O5,10.0,268.22608,9.0,ins
Single,l-alanine,Amino acid,C3H7NO2,3.0,89.09318,30.0,ala__L
Single,l-glutamate,Amino acid,NaC5H8NO4,5.0,169.11000,18.0,glu__L
Single,l-histidine,Amino acid,C6H9N3O2,6.0,155.15456,15.0,his__L


In [8]:
cs_to_conc_dict = df_cs.loc[:, ['Metabolite name', 'Concentration (mM)']].set_index('Metabolite name').to_dict()['Concentration (mM)']
# cs_to_bigg_dict = df_cs.loc[:, ['Metabolite name', 'Bigg ID']].set_index('Metabolite name').to_dict()['Bigg ID']

In [9]:
# model_ecoli = reframed.load_cbmodel('../../models/e_coli/iJO1366.xml')
# model_ecoli.metabolites['M_cm_e'].name

In [10]:
df_ex = pd.read_csv(FN_EXCHANGED_METS)

In [11]:
# m_names = []
# for m_id in df_ex['M id']:
#     try:
#         m = model_ecoli.metabolites[f'{m_id[:-1]+"c"}']
#     except KeyError:
#         m_names.append("")
#     else:
#         m_names.append(m.name)

In [12]:
# df_ex['Metabolite names'] = m_names

In [13]:
# df_ex.to_csv(FN_EXCHANGED_METS)

In [14]:
df_growth = pd.read_csv(FN_GROWTH_PARAMETERS, index_col=0)
df_growth['Carbon source'] = df_growth['Carbon source'].str.lower()

In [15]:
v_max_dict =  df_growth[['Species', 'Carbon source', 'max_growth_rate']].set_index(['Species', 'Carbon source']).to_dict()['max_growth_rate']

In [16]:
carbon_sources = [ x for x in df_growth['Carbon source'].unique() if x.lower() != 'no cs']


In [17]:
# v_max_dict = 
# for idx, row in df_growth.iterrows():
#     print(row['Carbon source'], row['Species'])


In [18]:
fraction_of_optimum= 0.9
D = dFBA(iterations = 30, dt = 0.5, method = "FBA_with_leakage", store_exchanges_flag = False, fraction_of_optimum=fraction_of_optimum)#(, fraction_of_optimum = 0.95)
cs = 'd-glucose'
conc = cs_to_conc_dict[cs]
cs_id = f'M_{cs_to_bigg_dict[cs]}_e'
D.medium.define_initial_conditions({cs_id: conc, 'M_thm_e': 1000})
INITIAL_OD = 0.03 # Per species
gDW_per_OD = 0.4 
initial_gDW = INITIAL_OD * gDW_per_OD


In [19]:
model_name_dict = {}
for strain in species:
    model_fn = f'../../models/mwf/polished_{strain}.xml'
    model = reframed.load_cbmodel(model_fn)
    model_name_dict[strain] = [model, initial_gDW]
D.add_models(model_name_dict)

Can't set Vmax for nh4_e; not in model At
Can't set Vmax for ca2_e; not in model At
Can't set Vmax for cobalt2_e; not in model At
Can't set Vmax for h_e; not in model At
Can't set Vmax for h2o_e; not in model At
Can't set Vmax for mg2_e; not in model At
Can't set Vmax for o2_e; not in model At
Can't set Vmax for pi_e; not in model At
Can't set Vmax for k_e; not in model At
Can't set Vmax for na1_e; not in model At
Can't set Vmax for so4_e; not in model At
Can't set Vmax for cu2_e; not in model At
Can't set Vmax for mn2_e; not in model At
Can't set Vmax for cl_e; not in model At
Can't set Vmax for fe3_e; not in model At
Can't set Vmax for zn2_e; not in model At
Can't set Vmax for ni2_e; not in model At
Can't set Vmax for fe2_e; not in model At
Can't set Vmax for mobd_e; not in model At
Can't set Vmax for btn_e; not in model At
Can't set Vmax for pqqA_kt_c; not in model At
Can't set Vmax for nh4_e; not in model Ct
Can't set Vmax for ca2_e; not in model Ct
Can't set Vmax for cobalt2_e; no

# Add info about predicted yield from metabolic models

In [20]:
def set_auxotrophy_dict(auxotrophy_dict):
    constraints_dict = {}
    for species_abbr, species_dict in auxotrophy_dict.items():
        species_constraint_dict = {}
        if species_dict.get('vitamins'):
            for met_id in species_dict['vitamins']:
                r_id = f'R_EX_{met_id}_e'
                species_constraint_dict[r_id] = (-0.001, 0)
        if species_dict.get('amino acids'):
            for met_id in species_dict['amino acids']:
                r_id = f'R_EX_{met_id}_e'
                species_constraint_dict[r_id] = (-0.2, 0)
        constraints_dict[species_abbr] = species_constraint_dict
    return constraints_dict

In [21]:
auxotrophy_dict = {
    'Ml': {'amino acids': ['pro__L', 'cys__L'], 'vitamins':['thm', 'btn']},
    'Oa': {'vitamins': ['thm']}
    }

auxotrophy_constraints = set_auxotrophy_dict(auxotrophy_dict)

In [22]:
auxotrophy_constraints

{'Ml': {'R_EX_thm_e': (-0.001, 0),
  'R_EX_btn_e': (-0.001, 0),
  'R_EX_pro__L_e': (-0.2, 0),
  'R_EX_cys__L_e': (-0.2, 0)},
 'Oa': {'R_EX_thm_e': (-0.001, 0)}}

In [23]:
# print(strain)
# uptake = -2
# constraints = {'R_EX_glc__D_e':uptake}
# if auxotrophy_constraints.get(strain):
#     constraints.update(auxotrophy_constraints[strain])
# sol = reframed.FBA(cb_model, constraints=constraints)
# theoretical_yield = sol.fobj/-uptake
# print(sol, theoretical_yield)

# Now add info about leakage and uptake

In [24]:
theoretical_yield_list = []
max_uptake_list = []
uptake = -2
for i, row in df_growth.iterrows():
    strain = row['Species']
    cs = row['Carbon source']
    if cs == 'no cs':
        theoretical_yield = 0
        max_uptake = 0
    else:
            
        cb_model = model_name_dict[strain][0]
        cs_id = cs_to_bigg_dict[cs]
        constraints = {f'R_EX_{cs_id}_e':uptake}
        if auxotrophy_constraints.get(strain):
            constraints.update(auxotrophy_constraints[strain])
        sol = reframed.FBA(cb_model, constraints=constraints)
        # print(strain, cs, sol.fobj)
        if sol.fobj:
            theoretical_yield = sol.fobj/-uptake
            max_uptake = row['max_growth_rate']/(theoretical_yield*fraction_of_optimum)
        else:
            theoretical_yield = 0
            max_uptake = 0
    theoretical_yield_list.append(theoretical_yield)
    max_uptake_list.append(max_uptake)
    
    # print(strain, cs, sol, theoretical_yield)
df_growth['Theoretical yield'] = theoretical_yield_list
df_growth['Max uptake'] = max_uptake_list

Set parameter Username


2024-04-26 11:52:31 INFO:gurobipy.gurobipy Set parameter Username


Academic license - for non-commercial use only - expires 2024-12-19


2024-04-26 11:52:31 INFO:gurobipy.gurobipy Academic license - for non-commercial use only - expires 2024-12-19


In [31]:
def log_leakage_rates_from_LM(model, leak_mets, leak_exchanges, shadow_prices, add_noise = False):
    log_leakage_rates = {}
    for i, (m_id, r_ex_id) in enumerate(zip(leak_mets, leak_exchanges)):
        bigg_id = m_id[2:-2]
        m = model.metabolites[m_id]
        charge = int(float(m.metadata['CHARGE']))
        log_intra_conc = bigg_to_log_intra_conc_dict[bigg_id]
        metabolite_value = np.log10(-shadow_prices[m_id])
        met_class = met_id_to_class[bigg_id]
        class_value = class_dict[met_class]
        log_leakage_rates[m_id] = LINEAR_MODEL_DICT['Intercept'] + class_value + LINEAR_MODEL_DICT["log10(Intracell. conc. [mM])"]*log_intra_conc + LINEAR_MODEL_DICT["log10(metabolite value)"]*metabolite_value + LINEAR_MODEL_DICT['charge']*charge
        if add_noise:
            log_leakage_rates[m_id] = log_leakage_rates[m_id] + np.random.normal(0, LINEAR_MODEL_DICT['model RMSE'])
    return log_leakage_rates

In [29]:
LINEAR_MODEL_DICT = {
    "Intercept":-4.730421332,
    "Compound class: Carboxylic acid":1.88106052,
    "Compound class: Nucleosides and nucleotides":0.125281202,
    "Compound class: Other":1.154873109,
    "Compound class: Phosphorylated sugar":0.84804023,
    "log10(Intracell. conc. [mM])":0.485531592,
    "log10(metabolite value)":-0.940301993,
    "charge":0.461502729,
    "model RMSE":0.73,

}
class_dict = defaultdict(lambda: 1.15)
class_dict['Carboxylic acid'] = 1.88
class_dict['Nucleosides and nucleotides'] = 0.13
class_dict['Phosphorylated sugar'] = 0.84

def predict_log_leakage_rates2(model, constraints, fraction_of_optimum):
    """
    Update on April 22
    """
    leak_mets, leak_exchanges = get_leaky_metabolites(model, constraints=constraints,
                                                      fraction_of_optimum=fraction_of_optimum)
    
    shadow_prices = estimate_shadow_prices(model, constraints=constraints,
                                                            metabolites = leak_mets)

    log_leakage_rates = log_leakage_rates_from_LM(model, 
                                leak_mets, leak_exchanges, shadow_prices, add_noise = True)
    return log_leakage_rates, leak_mets, leak_exchanges

In [25]:
max_uptake_dict = df_growth.set_index(['Species', 'Carbon source'])['Max uptake'].to_dict()
# max_uptake_dict

In [26]:
a = D.models['Ct']

In [27]:
a.set_Vmax('M_ac_e', 8)

In [32]:
cs_for_sp_estimation = {
    'At': 'glc__D',
    'Ct': 'ac',
    'Ml': 'glc__D',
    'Oa': 'glc__D',
}
for strain in species:
    # Uptake
    print(strain)
    model = D.models[strain]
    constraints = {f'R_EX_{cs_for_sp_estimation[strain]}_e': -5}
    if auxotrophy_constraints.get(strain):
        constraints.update(auxotrophy_constraints[strain])
    log_leakage_rates, leak_mets, leak_exchanges = predict_log_leakage_rates2(model.cbmodel, constraints, fraction_of_optimum)
    # print(log_leakage_rates)
    for key, value in log_leakage_rates.items():
        log_leakage_rates[key] =  value + 1
    model.log_leakage_rates = log_leakage_rates
    model.leak_mets = leak_mets
    model.leak_exchanges = leak_exchanges
    for cs in carbon_sources:
        m_id = cs_to_bigg_dict[cs]
        model.set_Vmax(f'M_{m_id}_e', max_uptake_dict[(strain, cs)])


At
Candidate metabolites 57:
['M_2pg_c', 'M_3pg_c', 'M_4hba_c', 'M_5drib_c', 'M_ac_c', 'M_acald_c', 'M_adn_c', 'M_akg_c', 'M_ala__L_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_btn_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 'M_f6p_c', 'M_fald_c', 'M_fdp_c', 'M_fe3dcit_c', 'M_for_c', 'M_fum_c', 'M_g3p_c', 'M_g6p_c', 'M_gln__L_c', 'M_glu__L_c', 'M_gly_c', 'M_glyc3p_c', 'M_his__L_c', 'M_hom__L_c', 'M_icit_c', 'M_ile__L_c', 'M_inost_c', 'M_leu__L_c', 'M_lys__L_c', 'M_mal__L_c', 'M_meoh_c', 'M_met__L_c', 'M_oaa_c', 'M_orn_c', 'M_orot_c', 'M_pep_c', 'M_phe__L_c', 'M_ppa_c', 'M_pro__L_c', 'M_pyr_c', 'M_r5p_c', 'M_ru5p__D_c', 'M_ser__L_c', 'M_skm_c', 'M_succ_c', 'M_thr__L_c', 'M_trp__L_c', 'M_tyr__L_c', 'M_val__L_c', 'M_xu5p__D_c']
Ct
Candidate metabolites 53:
['M_2pg_c', 'M_34dhbz_c', 'M_3pg_c', 'M_4hba_c', 'M_4hbz_c', 'M_ac_c', 'M_adn_c', 'M_akg_c', 'M_ala_B_c', 'M_ala__L_c', 'M_alaala_c', 'M_arg__L_c', 'M_asn__L_c', 'M_asp__L_c', 'M_cit_c', 'M_cys__L_c', 'M_dhap_c', 'M_e4p_c', 

In [33]:
for strain in species:
    model = D.models[strain]
    model.initiate_cobra_specific_model(auxotrophy_constraints=auxotrophy_constraints)

2024-04-26 11:53:28 INFO:Model At Use FBA with leakage
2024-04-26 11:53:28 INFO:Model Ct Use FBA with leakage
2024-04-26 11:53:28 INFO:Model Ml Use FBA with leakage
2024-04-26 11:53:29 INFO:Model Oa Use FBA with leakage


In [34]:
D.run()

2024-04-26 11:53:30 INFO:dFBA     Model At exchanges: {'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_14glucan_e': 0.0, 'R_EX_23camp_e': 0.0, 'R_EX_23ccmp_e': 0.0, 'R_EX_23cgmp_e': 0.0, 'R_EX_23cump_e': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3amp_e': 0.0, 'R_EX_3cmp_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_3ump_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hba_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hppta_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt_e': 0.0, 'R_EX_ac_e': 0.0, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ad_e': 0.0, 'R_EX_adn_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_alahis_e': 0.0, 'R_EX_alaleu_e': 0.0, 'R_EX_alathr_e': 0.0, 'R_E

0.21720239513155454 0.1954821556183991 0.9
At {'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_14glucan_e': 0.0, 'R_EX_23camp_e': 0.0, 'R_EX_23ccmp_e': 0.0, 'R_EX_23cgmp_e': 0.0, 'R_EX_23cump_e': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3amp_e': 0.0, 'R_EX_3cmp_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_3ump_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hba_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hppta_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt_e': 0.0, 'R_EX_ac_e': 0.0, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ad_e': 0.0, 'R_EX_adn_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_alahis_e': 0.0, 'R_EX_alaleu_e': 0.0, 'R_EX_alathr_e': 0.0, 'R_EX_alatrp

2024-04-26 11:53:30 INFO:dFBA     Model Ml exchanges: {'R_sink_2ohph_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2ddglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_34dhpac_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hphac_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_4oxptn_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6hnac_e': 0.0, 'R_EX_LalaDgluMdapDala_e': 0.0, 'R_EX_LalaDgluMdap_e': 0.0, 'R_EX_LalaDglu_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_Lcyst_e': 0.0, 'R_EX_R3hdec4e_e': 0.0, 'R_EX_R_3hphpa_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_ac_e': 1.031056236255657, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acmana_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_acon_C_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_B_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala_L_thr__L_e': 0.0, 'R_EX_ala__D_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_ala_gln_e': 0.0, 'R_EX_ala_his_e': 0.0, 'R_EX_ala_leu_e':

0.37441204517459037 0.33697084065713134 0.9
Oa {'R_sink_2ohph_c': 0.0, 'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3oxoadp_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4abzglu_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 1.3478833626285256e-06, 'R_EX_5mdru1p_e': 0.0, 'R_EX_5mtr_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R3hdec4e_e': 0.0, 'R_EX_R_3h4atba_e': 0.0, 'R_EX_R_3hhpa_e': 0.0, 'R_EX_R_3hpdeca_e': 0.0, 'R_EX_R_3hphpa_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hpocta_e': 0.0, 'R_EX_R_3hpt_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt__L_e': 0.0, 'R_EX_ac_e': -0.15500033862614804, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acmana_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ade_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_B_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__

2024-04-26 11:53:30 INFO:dFBA     Model Oa exchanges: {'R_sink_2ohph_c': 0.0, 'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3oxoadp_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4abzglu_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 1.2534773568301367e-06, 'R_EX_5mdru1p_e': 0.0, 'R_EX_5mtr_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R3hdec4e_e': 0.0, 'R_EX_R_3h4atba_e': 0.0, 'R_EX_R_3hhpa_e': 0.0, 'R_EX_R_3hpdeca_e': 0.0, 'R_EX_R_3hphpa_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hpocta_e': 0.0, 'R_EX_R_3hpt_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt__L_e': 0.0, 'R_EX_ac_e': 0.0, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acmana_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ade_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_B_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__L_e': 0.0,

...

0.22349438140310945 0.20114494326279853 0.9
At {'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_14glucan_e': 0.0, 'R_EX_23camp_e': 0.0, 'R_EX_23ccmp_e': 0.0, 'R_EX_23cgmp_e': 0.0, 'R_EX_23cump_e': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3amp_e': 0.0, 'R_EX_3cmp_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_3ump_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hba_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hppta_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt_e': 0.0, 'R_EX_ac_e': 0.0, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ad_e': 0.0, 'R_EX_adn_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_alahis_e': 0.0, 'R_EX_alaleu_e': 0.0, 'R_EX_alathr_e': 0.0, 'R_EX_alatr

2024-04-26 11:53:37 INFO:dFBA     Model Ml exchanges: {'R_sink_2ohph_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2ddglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_34dhpac_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hphac_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_4oxptn_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6hnac_e': 0.0, 'R_EX_LalaDgluMdapDala_e': 0.0, 'R_EX_LalaDgluMdap_e': 0.0, 'R_EX_LalaDglu_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_Lcyst_e': 0.0, 'R_EX_R3hdec4e_e': 0.0, 'R_EX_R_3hphpa_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_ac_e': 0.7807559861365776, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acmana_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_acon_C_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_B_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala_L_thr__L_e': 0.0, 'R_EX_ala__D_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_ala_gln_e': 0.0, 'R_EX_ala_his_e': 0.0, 'R_EX_ala_leu_e'

0.07489119752578781 0.06740207777320903 0.9
Ct {'R_sink_2ohph_c': 0.0, 'R_sink_4crsol_c': 0.0, 'R_sink_amob_c': 9.913938333953271e-05, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_15dap_e': 0.0, 'R_EX_23camp_e': 0.0, 'R_EX_23ccmp_e': 0.0, 'R_EX_23cgmp_e': 0.0, 'R_EX_23cump_e': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2ameph_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_2pglyc_e': 0.0, 'R_EX_34dhbz_e': 0.0, 'R_EX_34dhcinm_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3amp_e': 0.0, 'R_EX_3cmp_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3hcinnm_e': 0.0, 'R_EX_3hpppn_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_3ump_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4hba_e': 0.0, 'R_EX_4hbz_e': 0.0, 'R_EX_4hphac_e': 0.0, 'R_EX_4hptn_e': 0.0, 'R_EX_4oxptn_e': 0.0, 'R_EX_5aptn_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_5mdru1p_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_6hnac_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_Lcyst_e': 0.0, 'R_EX_R_3hdcaa_e': 0.0, 'R_EX_R_3hhdca_e': 0.0, 'R_EX_R_3hhxa_e': 0.0, 'R_EX_R_3hocta_e': 0.0, 'R_EX_R

2024-04-26 11:53:37 INFO:dFBA     Model Ml exchanges: {'R_sink_2ohph_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2ddglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_34dhpac_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hphac_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_4oxptn_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6hnac_e': 0.0, 'R_EX_LalaDgluMdapDala_e': 0.0, 'R_EX_LalaDgluMdap_e': 0.0, 'R_EX_LalaDglu_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_Lcyst_e': 0.0, 'R_EX_R3hdec4e_e': 0.0, 'R_EX_R_3hphpa_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_ac_e': 1.0322602405071657, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acmana_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_acon_C_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_B_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala_L_thr__L_e': 0.0, 'R_EX_ala__D_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_ala_gln_e': 0.0, 'R_EX_ala_his_e': 0.0, 'R_EX_ala_leu_e'

0.22576032101821197 0.2031842889163908 0.9
At {'R_sink_4crsol_c': 0.0, 'R_sink_hemeO_c': 0.0, 'R_sink_mobd_c': 0.0, 'R_sink_sheme_c': 0.0, 'R_EX_14glucan_e': 0.0, 'R_EX_23camp_e': 0.0, 'R_EX_23ccmp_e': 0.0, 'R_EX_23cgmp_e': 0.0, 'R_EX_23cump_e': 0.0, 'R_EX_25dkglcn_e': 0.0, 'R_EX_2m35mdntha_e': 0.0, 'R_EX_35dnta_e': 0.0, 'R_EX_3amp_e': 0.0, 'R_EX_3cmp_e': 0.0, 'R_EX_3h4atb_e': 0.0, 'R_EX_3mb_e': 0.0, 'R_EX_3ump_e': 0.0, 'R_EX_4abut_e': 0.0, 'R_EX_4ahmmp_e': 0.0, 'R_EX_4hba_e': 0.0, 'R_EX_4hpro_LT_e': 0.0, 'R_EX_5drib_e': 0.0, 'R_EX_6atha_e': 0.0, 'R_EX_LalaLglu_e': 0.0, 'R_EX_R_3hpnona_e': 0.0, 'R_EX_R_3hppta_e': 0.0, 'R_EX_abg4_e': 0.0, 'R_EX_abt_e': 0.0, 'R_EX_ac_e': 0.0, 'R_EX_acald_e': 0.0, 'R_EX_acgam_e': 0.0, 'R_EX_acnam_e': 0.0, 'R_EX_actn__R_e': 0.0, 'R_EX_ad_e': 0.0, 'R_EX_adn_e': 0.0, 'R_EX_agm_e': 0.0, 'R_EX_akg_e': 0.0, 'R_EX_ala_L_asp__L_e': 0.0, 'R_EX_ala_L_glu__L_e': 0.0, 'R_EX_ala__L_e': 0.0, 'R_EX_alahis_e': 0.0, 'R_EX_alaleu_e': 0.0, 'R_EX_alathr_e': 0.0, 'R_EX_alatrp

In [37]:
D.concentrations_df['M_ac_e']

0     0.000000
1     0.008205
2     0.001858
3     0.001439
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.009450
10    0.000000
11    0.000000
12    0.000000
13    0.009450
14    0.007165
15    0.007433
16    0.000873
17    0.009450
18    0.001746
19    0.000000
20    0.000000
21    0.000000
22    0.006188
23    0.000057
24    0.004610
25    0.004684
26    0.006191
27    0.004685
28    0.006192
29    0.000000
Name: M_ac_e, dtype: float64

In [ ]:
log_leakage_rates

In [ ]:
np.random.normal?